In [ ]:
import os
import pandas as pd
import re
from cobra.io import load_json_model

MODEL_JSON = 'Accumulibacter_anaerobic.json'
SUMMARY_FILE = 'stoichiometry_pathways.csv'

PATHWAYS = {
    'NTS': {
        'bounds': [
            ('GAPDH', 0, 'both'),
            ('ACS',   0, 'lower'),
            ('GlycP', 1, 'both'),
        ],
        'rxn_order': [
            'GlycP','GPM','PGI','ATPPFK','FBPald','TPI',
            'TktA','SBPald','SBPase','TktB','RibE','RibI',
            'RbuK','RbuCO','PGM','ENO','PYK','PDH','ACS',
            'Thio','AAR','PHBsyn'
        ]
    },
    'EMP': {
        'bounds': [
            ('TktA', 0, 'both'),
            ('ACS',   0, 'lower'),
            ('GAPDH', 0, 'lower'),
            ('GlycP', 1, 'both'),
        ],
        'rxn_order': [
            'GlycP','GPM','PGI','ATPPFK','FBPald','TPI',
            'GAPDH','PGK','PGM','ENO','PYK','PDH','ACS',
            'Thio','AAR','PHBsyn'
        ]
    }
}

def delete_file(path):
    if os.path.isfile(path):
        os.remove(path)
        print(f'Deleted old file: {path}')
    else:
        print(f'No existing file to delete: {path}')

delete_file(SUMMARY_FILE)


def standardize_arrow(formula: str) -> str:
    # Replace any arrow (-> or <=>) with <=>
    return re.sub(r"\s*(?:<=>|-->)\s*", " <=> ", formula)

# Process results function
def process_fba_results(model, solution, pathway_name, rxn_order):
    # extract exchange fluxes
    ex_map = {
        'Ace':'EX_Ace','CO2':'EX_CO2','Mal3':'EX_Mal3','Mal4':'EX_Mal4',
        'HB':'EX_HB','PHB':'EX_PHB','H2O':'EX_H2O','PolyPP':'EX_PolyPP',
        'PolyP':'EX_PolyP','Pi':'EX_Pi','PPi':'EX_PPi'
    }
    ex_fluxes = {met: solution.fluxes[rxn_id] for met, rxn_id in ex_map.items()}

    # update summary table
    row = {'pathway': pathway_name, **ex_fluxes}
    if os.path.isfile(SUMMARY_FILE):
        df_sum = pd.read_csv(SUMMARY_FILE)
        if pathway_name in df_sum['pathway'].values:
            df_sum.loc[df_sum['pathway']==pathway_name, list(ex_fluxes.keys())] = list(ex_fluxes.values())
        else:
            df_sum = pd.concat([df_sum, pd.DataFrame([row])], ignore_index=True)
    else:
        df_sum = pd.DataFrame([row])
    df_sum.to_csv(SUMMARY_FILE, index=False)

    # full flux table with reversible formulas
    full = pd.DataFrame({
        'Reaction Name': [r.id for r in model.reactions],
        'Reaction Formula': [standardize_arrow(r.reaction) for r in model.reactions],
        'Relative Flux': solution.fluxes.values
    })
    full.to_csv(f'{pathway_name}_full.csv', index=False)

    # filtered: remove zero flux and exchanges
    nonzero = full[full['Relative Flux'].abs() > 1e-6]
    nonexchange = nonzero[~nonzero['Reaction Name'].str.startswith('EX_')]
    filtered = nonexchange.set_index('Reaction Name').reindex(rxn_order).dropna().reset_index()
    filtered.to_csv(f'{pathway_name}.csv', index=False)

# Main FBA loop
for pathway_name, cfg in PATHWAYS.items():
    print(f'=== {pathway_name} ===')
    model = load_json_model(MODEL_JSON)
    # apply bounds
    for rxn_id, bound, btype in cfg['bounds']:
        rxn = model.reactions.get_by_id(rxn_id)
        if btype in ('lower','both'): rxn.lower_bound = bound
        if btype in ('upper','both'): rxn.upper_bound = bound
    # run FBA
    sol = model.optimize()
    print(f"Pathway: {pathway_name}, Status: {sol.status}, Objective: {sol.objective_value}")
    if sol.status != 'optimal':
        print(f"WARNING: FBA for {pathway_name} was infeasible.")
        continue  # or raise an error, or write zeroed outputs intentionally
    process_fba_results(model, sol, pathway_name, cfg['rxn_order'])
    # print full flux vector
    for r, f in zip(model.reactions, sol.fluxes.values):
        print(f"{r.id}: {f:.6f}")
